In [2]:
# Dependencies 
import pandas as pd
import numpy as np

In [3]:
# File to load 
census_df = pd.read_csv ("census_pop_density.csv")

In [4]:
import unicodedata
currentFile = open("NCHS.csv", "rt",encoding="utf-8")

In [5]:
NCHS_df = pd.read_csv (currentFile)

In [6]:
# Cleaning Census Data
census_df.head()
for col in census_df.columns:
    print(col)

OBJECTID
COUNTYNS
GEOID
ALAND
AWATER
NAME
State
B25010_001E
B25010_001M
B25010_002E
B25010_002M
B25010_003E
B25010_003M
B01001_001E
B01001_001M
B01001_calc_PopDensity
created_user
created_date
last_edited_user
last_edited_date
B01001_calc_PopDensityM
SHAPE_Length
SHAPE_Area


In [7]:
census_df=census_df.drop(columns=["GEOID","OBJECTID", "COUNTYNS", "ALAND", "AWATER","B25010_002E",
"B25010_002M", "B25010_003E", "B25010_003M","B01001_001M", "created_user", "created_date",
"last_edited_user", "last_edited_date", "B01001_calc_PopDensityM", "SHAPE_Length","SHAPE_Area","B25010_001M"])

In [8]:
census_df=census_df.rename(columns={"NAME": "county","State":"state","B25010_001E":"avg_household","B01001_calc_PopDensity":"pop_dens(/sqmi)","B01001_001E":"total_pop" })
census_df.head()
census_df= census_df.drop_duplicates()

In [9]:
# Cleaning NHCS Data
NCHS_df.head()

,FIPS code,State Abr.,County name,County 2012 pop,2013 code,2006 code,1990-based code
0,1001,AL,Autauga County,55514,3,3,3
1,1003,AL,Baldwin County,190790,4,5,3
2,1005,AL,Barbour County,27201,6,5,5
3,1007,AL,Bibb County,22597,2,2,6
4,1009,AL,Blount County,57826,2,2,3


In [10]:
NCHS_df=NCHS_df.drop(columns={"County 2012 pop","2006 code", "1990-based code","FIPS code","State Abr."})

In [11]:

NCHS_df.head()

,County name,2013 code
0,Autauga County,3
1,Baldwin County,4
2,Barbour County,6
3,Bibb County,2
4,Blount County,2


In [12]:
NCHS_df=NCHS_df.rename(columns={"County name":"county","2013 code":"nchs_code_2013"})
NCHS_df= NCHS_df.drop_duplicates("county")

In [13]:
# Combining the dataframes
county_pop_df=pd.merge(census_df,NCHS_df,how="inner",on=["county"])

In [14]:
county_pop_df.head()

,county,state,avg_household,total_pop,pop_dens(/sqmi),nchs_code_2013
0,Autauga County,Alabama,2.59,55200,35.853419,3
1,Baldwin County,Alabama,2.61,208107,50.541504,4
2,Baldwin County,Georgia,2.54,45286,67.588266,4
3,Barbour County,Alabama,2.49,25782,11.247981,6
4,Barbour County,West Virginia,2.52,16730,18.939514,6


In [15]:
county_pop_df=county_pop_df.loc[(county_pop_df["nchs_code_2013"]<=2)&(county_pop_df["pop_dens(/sqmi)"]>1000)]
county_pop_df=county_pop_df.drop_duplicates(ignore_index=True)
county_pop_df.head(10)

,county,state,avg_household,total_pop,pop_dens(/sqmi),nchs_code_2013
0,Orange County,California,3.02,3164182,1541.026670,1
1,San Francisco County,California,2.36,870044,7168.190471,1
2,Denver County,Colorado,2.31,693417,1746.533667,1
3,Middlesex County,New Jersey,2.80,826698,1032.248366,2
4,District of Columbia,District of Columbia,2.29,684498,4322.952569,1
5,Nassau County,New York,3.00,1356564,1840.443984,2
6,Pinellas County,Florida,2.31,957875,1351.009895,1
7,DuPage County,Illinois,2.69,931743,1097.706801,2
8,Baltimore city,Maryland,2.48,614700,2932.040761,1
9,Essex County,New Jersey,2.73,793555,2430.062385,2


In [16]:
county_pop_df.dropna(inplace = True)


In [17]:
county_pop_df['county'] = county_pop_df['county'].str.rsplit(" ", n = 1, expand=True)[0]
county_pop_df.head()

,county,state,avg_household,total_pop,pop_dens(/sqmi),nchs_code_2013
0,Orange,California,3.02,3164182,1541.026670,1
1,San Francisco,California,2.36,870044,7168.190471,1
2,Denver,Colorado,2.31,693417,1746.533667,1
3,Middlesex,New Jersey,2.80,826698,1032.248366,2
4,District of,District of Columbia,2.29,684498,4322.952569,1


In [18]:
county_pop_df['county']=county_pop_df['county'].apply(str.lower)
county_pop_df['state']=county_pop_df['state'].apply(str.lower)

In [19]:
county_pop_df.to_csv('county_pop_final.csv',header=True, index=True)

In [20]:
dense_county_pop_df=county_pop_df.loc[(county_pop_df["pop_dens(/sqmi)"]>=3000)]
sparse_county_pop_df=county_pop_df.loc[(county_pop_df["pop_dens(/sqmi)"]<=2999)]

In [21]:
dense_county_pop_df.to_csv('dense_county_pop.csv',header=True, index=True)
sparse_county_pop_df.to_csv('sparse_county_pop.csv',header=True, index=True)